Numpy based CART, incuding integration of the tree

In [181]:
import pickle
import numpy as np
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
import math
from sklearn.ensemble import GradientBoostingRegressor

In [182]:
def binarySplit(dataset,feature,value):
    matLeft = dataset[dataset[:, feature] <= value]
    matRight = dataset[dataset[:, feature] > value]
    return matLeft, matRight

In [183]:
def regressLeaf(dataset):
    if len(dataset) == 0: return 0
    return np.mean(dataset[:,-1])

In [184]:
def getError(dataset,featureIndex,thresholdSamples):
    dataset = dataset[dataset[:,featureIndex].argsort()]
    tot_sum = sum(dataset[:,-1])
    tot_sum_square = sum(dataset[:,-1]*dataset[:,-1])
    cur = 0
    tmpVariance = float('inf')
    tmpIndex = thresholdSamples
    for i in range(dataset.shape[0]-thresholdSamples):
        cur += dataset[i][-1]
        variance = -cur*cur/(i+1)-(tot_sum-cur)*(tot_sum-cur)/(dataset.shape[0]-i-1)
        if variance < tmpVariance and i > thresholdSamples:
            tmpVariance = variance
            tmpIndex = i
    return tot_sum_square+tmpVariance, dataset[tmpIndex][featureIndex]

In [487]:
def chooseBestSplit(dataset,leafType=regressLeaf,threshold=(0,4)):
    thresholdErr, thresholdSamples = threshold[0], threshold[1]
    if len(set(dataset[:,-1])) == 1 or dataset.shape[0]<=thresholdSamples:
        return None,leafType(dataset)
    m,n = dataset.shape
    Err = float('inf')
    bestErr, bestFeatureIndex, bestFeatureValue = np.inf, 0, 0
    for featureIndex in range(n-1):
        tmpErr, featureValue = getError(dataset,featureIndex,thresholdSamples)
        if tmpErr < bestErr:
            bestErr,bestFeatureIndex,bestFeatureValue = tmpErr,featureIndex,featureValue
    if (Err - bestErr) < thresholdErr:
        return None, leafType(dataset)
    return bestFeatureIndex,bestFeatureValue

In [488]:
def createCART(dataset,leafType=regressLeaf,threshold=(1,4),depth=5):
    if depth==0: return leafType(dataset)
    feature,value = chooseBestSplit(dataset,leafType,threshold)
    if feature == None: return value
    returnTree = {}
    returnTree['bestSplitFeature'] = feature
    returnTree['bestFeatureValue'] = value
    leftSet, rightSet = binarySplit(dataset,feature,value)
    returnTree['left'] = createCART(leftSet,leafType,threshold,depth-1)
    returnTree['right'] = createCART(rightSet,leafType,threshold,depth-1)
    return returnTree

In [489]:
def isTree(obj):
        return (type(obj).__name__=='dict')

In [490]:
def regressEvaluation(tree,inputData):
    return float(tree)

In [491]:
def valuePredict(tree,inputData,modelEval = regressEvaluation):
        if not isTree(tree): return modelEval(tree,inputData)
        if inputData[tree['bestSplitFeature']] <= tree['bestFeatureValue']:
                if isTree(tree['left']):
                        return valuePredict(tree['left'],inputData,modelEval)
                else:
                        return modelEval(tree['left'],inputData)
        else:
                if isTree(tree['right']):
                        return valuePredict(tree['right'],inputData,modelEval)
                else:
                        return modelEval(tree['right'],inputData)

In [492]:
def predict(tree,testData,modelEval=regressEvaluation):
        m = len(testData)
        yHat = []
        for i in range(m):
                yHat += [valuePredict(tree,testData[i],modelEval)]
        return np.array(yHat)

In [493]:
def tree_evaluator(tree,interval):
    if (type(tree).__name__!='dict'):
# check if it's nan
        if tree!= tree:
            return 0
        res = tree
        for i in interval:
            res *= (interval[i][1] - interval[i][0])
#        print(interval,tree)
        return res
    else:
        axis = tree['bestSplitFeature']
        val = tree['bestFeatureValue']
        left_interval = {key:[i for i in interval[key]] for key in interval}
        right_interval = {key:[i for i in interval[key]] for key in interval}
        if axis in interval:
            left_interval[axis][1] = min(left_interval[axis][1],val)
            right_interval[axis][0] = max(right_interval[axis][0],val)
        else:
            left_interval[axis] = [0,val]
            right_interval[axis] = [val,1]
        return tree_evaluator(tree['left'],left_interval) + tree_evaluator(tree['right'],right_interval)

In [494]:
def Integrand(grid):
    res = []
    for var in grid:
        x,y,z,w,a,b = var
        res += [x+y-z+np.exp(w)-np.tan(a)+np.cos(b)]
    return np.array(res)

In [495]:
grid = np.random.rand(10000,6)
vals = Integrand(grid)

In [496]:
dataset = []
for i in range(len(grid)):
    tmp = grid[i].tolist() + [vals[i]]
    dataset += [tmp]
dataset = np.array(dataset)

In [497]:
tree_depth = 5

In [498]:
%%time
regressTree = createCART(dataset,depth=tree_depth)
y_hat = predict(regressTree,grid)


res = 0
for i in range(len(y_hat)):
    res += (y_hat[i] - vals[i])**2
print(res/len(y_hat))

0.1765301815746734
CPU times: user 471 ms, sys: 955 µs, total: 472 ms
Wall time: 470 ms


In [499]:
tree_evaluator(regressTree,{})

2.4442215204366615

In [198]:
%%time
regressor = DecisionTreeRegressor(random_state=0,max_depth=tree_depth)
regressor.fit(grid,vals)

y_hat = regressor.predict(grid)
res = 0
for i in range(len(y_hat)):
    res += (y_hat[i] - vals[i])**2
print(res/len(y_hat))

0.17781692626550416
CPU times: user 34.1 ms, sys: 1 ms, total: 35.1 ms
Wall time: 33.1 ms


In [454]:
def tree_integrate(estimator):
    feature = estimator.tree_.feature
    threshold = estimator.tree_.threshold
    value = estimator.tree_.value.flatten()
    return sk_tree_evalutaor(feature,threshold,value,{})

def sk_tree_evalutaor(feature,threshold,value,interval):
    if feature[0] < 0:
        res = value[0]
        for i in interval:
            res *= (interval[i][1]-interval[i][0])
        return res
    else:
        axis = feature[0]
        val = threshold[0]
        left_interval = {key:[i for i in interval[key]] for key in interval}
        right_interval = {key:[i for i in interval[key]] for key in interval}
        if axis in interval:
            left_interval[axis][1] = min(left_interval[axis][1],val)
            right_interval[axis][0] = max(right_interval[axis][0],val)
        else:
            left_interval[axis] = [0,val]
            right_interval[axis] = [val,1]
        if len(feature) == 3:
            loc = 2
        else:
            for i in range(1,len(feature)):
                if check_full(feature[1:i]) and check_full(feature[i:]):
                    loc = i
                    break
        left = sk_tree_evalutaor(feature[1:loc],threshold[1:loc],value[1:loc],left_interval)
        right = sk_tree_evalutaor(feature[loc:],threshold[loc:],value[loc:],right_interval)
        return left+right

In [455]:
def check_full(tree_node):
    if len(tree_node) == 1 and tree_node[0] == -2:
        return True
    stack = []
    for i in tree_node:
        if i >= 0: 
            stack += [i]
        else:
            if len(stack) >=1 and stack[-1] == -1:
                stack[-1] = -2
            elif len(stack) >= 1:
                stack[-1] = -1
        while len(stack) > 1 and stack[-1]==-2:
            if stack[-2] != -1:
                stack[-2] = -1
            else:
                stack[-2] = -2
            stack.pop()
    return stack == [-2]

In [436]:
tree_integrate(regressor)

2.4402987242789997

In [437]:
import pydotplus
dot_data = tree.export_graphviz(regressor, out_file=None)
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

AttributeError: 'DecisionTreeRegressor' object has no attribute 'export_graphviz'

numpy based BDT

In [201]:
def BDT(dataset,depth=2,n_est=10,lr=0.1):
    res = []
    running_dataset = np.copy(dataset)
    residue = np.copy(dataset)
    for _ in range(n_est):
        new_tree = createCART(running_dataset,depth=depth)
        y_hat = predict(new_tree,dataset[:,:-1])
        residue[:,-1] -= y_hat
        running_dataset[:,-1] = residue[:,-1]*lr
        res += [new_tree]
        if _ % (n_est//10) == 0:
            print('{} estimator trained'.format(_))
    return res

In [202]:
boosted_tree = BDT(dataset)

0 estimator trained
1 estimator trained
2 estimator trained
3 estimator trained
4 estimator trained
5 estimator trained
6 estimator trained
7 estimator trained
8 estimator trained
9 estimator trained


In [203]:
y_hat = sum([predict(tree,grid) for tree in boosted_tree])
res = 0
for i in range(len(y_hat)):
    res += (y_hat[i] - vals[i])**2
print(res/len(y_hat))

0.22773115694484686


In [204]:
sum([tree_evaluator(tree,{}) for tree in boosted_tree if tree_evaluator(tree,{})])

2.4393687841237144

In [205]:
f=open("screen.out", "r")
lines = f.readlines()

In [206]:
data = []
for line in lines[:-7]:
    nums = line.split()
    if len(nums) == 12:
        nums = [float(num) for num in nums]
        data += [nums]

In [207]:
data = np.array(data)

In [208]:
data

array([[6.28871e-01, 3.64784e-01, 5.13401e-01, ..., 1.63006e-02,
        2.42887e-01, 1.07647e+04],
       [1.37232e-01, 8.04177e-01, 1.56679e-01, ..., 8.39112e-01,
        6.12640e-01, 0.00000e+00],
       [2.96032e-01, 6.37552e-01, 5.24287e-01, ..., 4.00229e-01,
        8.91529e-01, 1.63409e+04],
       ...,
       [6.01785e-01, 7.35004e-02, 3.37025e-01, ..., 2.73972e-01,
        7.38237e-01, 7.45169e+01],
       [4.73995e-01, 6.29764e-02, 3.57536e-01, ..., 2.87283e-01,
        6.27453e-01, 0.00000e+00],
       [8.01884e-01, 1.79018e-01, 2.66675e-01, ..., 6.09341e-01,
        8.06570e-01, 0.00000e+00]])

In [46]:
boosted_tree = BDT(data,depth=2)

0 estimator trained
1 estimator trained
2 estimator trained
3 estimator trained
4 estimator trained
5 estimator trained
6 estimator trained
7 estimator trained
8 estimator trained
9 estimator trained


In [47]:
sum([tree_evaluator(tree,{}) for tree in boosted_tree if tree_evaluator(tree,{})])

8733.35587401102

In [48]:
tree_evaluator(boosted_tree[0],{})

8734.13274208207

In [49]:
boosted_tree[0]

{'bestSplitFeature': 5,
 'bestFeatureValue': 0.103537,
 'left': {'bestSplitFeature': 5,
  'bestFeatureValue': 0.0648196,
  'left': 611.0241823982898,
  'right': 4148.542707058089},
 'right': {'bestSplitFeature': 5,
  'bestFeatureValue': 0.894761,
  'left': 10521.312480372719,
  'right': 1987.7675159590021}}

In [177]:
def MC_GBDT_numpy(data_set,ratio):
    x = data_set[:,:-1]
    y = data_set[:,-1]
    split = int(ratio * x.shape[0])
    x_train, x_test, y_train, y_test = x[:split,:], x[split:,:] , y[:split], y[split:]
    
    boosted_tree = BDT(data_set[:split,:],depth=5,n_est=300,lr=0.2)
    y_hat = sum([predict(tree,x_test) for tree in boosted_tree])
    
    residue = y_test - y_hat
    residue_val = sum(residue)/len(residue)
    err = math.sqrt((sum(residue*residue)/len(residue)-residue_val*residue_val)/len(residue))
    print("Current error is {}, number of calls of integrand is {}".format(err,len(residue)))
    return err, residue_val+sum([tree_evaluator(tree,{}) for tree in boosted_tree if tree_evaluator(tree,{})])

In [180]:
MC_GBDT_numpy(data,ratio=0.2)

0 estimator trained
30 estimator trained
60 estimator trained
90 estimator trained
120 estimator trained
150 estimator trained
180 estimator trained
210 estimator trained
240 estimator trained
270 estimator trained
Current error is 2.6592227459981914, number of calls of integrand is 162000


(2.6592227459981914, 8724.503852596315)

In [234]:
def BDT_sklearn(dataset,depth=2,n_est=1000,lr=0.1):
    res = []
    running_dataset = np.copy(dataset)
    residue = np.copy(dataset)
    for _ in range(n_est):
        regressor = DecisionTreeRegressor(random_state=0,max_depth=depth)
        regressor.fit(running_dataset[:,:-1],running_dataset[:,-1])

        y_hat = regressor.predict(running_dataset[:,:-1])
        residue[:,-1] -= y_hat
        running_dataset[:,-1] = residue[:,-1]*lr
        res += [regressor]
        if _ % (n_est//10) == 0:
            print('{} estimator trained'.format(_))
    return res

In [235]:
boosted_tree = BDT_sklearn(data[:10000,:],depth=2)

0 estimator trained
100 estimator trained
200 estimator trained
300 estimator trained
400 estimator trained
500 estimator trained
600 estimator trained
700 estimator trained
800 estimator trained
900 estimator trained


In [461]:
def MC_GBDT(data_set,ratio):
    x = data_set[:,:-1]
    y = data_set[:,-1]
    split = int(ratio * x.shape[0])
    x_train, x_test, y_train, y_test = x[:split,:], x[split:,:] , y[:split], y[split:]
    boosted_tree = BDT_sklearn(data_set[:split,:],depth=6,n_est=100,lr=0.4)
    y_hat = sum([tree.predict(x_test) for tree in boosted_tree])

    residue = y_test - y_hat
    residue_val = sum(residue)/len(residue)
    err = math.sqrt((sum(residue*residue)/len(residue)-residue_val*residue_val)/len(residue))
    print("Current error is {}, number of calls of integrand is {}".format(err,len(residue)))
    return err, residue_val+sum([tree_integrate(tree) for tree in boosted_tree if tree_integrate(tree)])

In [468]:
MC_GBDT(data,ratio=0.05)

0 estimator trained
10 estimator trained
20 estimator trained
30 estimator trained
40 estimator trained
50 estimator trained
60 estimator trained
70 estimator trained
80 estimator trained
90 estimator trained
Current error is 3.402054312315024, number of calls of integrand is 192375


(3.402054312315024, 8725.176081504564)

In [473]:
rat = [0.01* i for i in range(1,20)]
for per in rat:
    MC_GBDT(data,ratio=per)

0 estimator trained
10 estimator trained
20 estimator trained
30 estimator trained
40 estimator trained
50 estimator trained
60 estimator trained
70 estimator trained
80 estimator trained
90 estimator trained
Current error is 4.573727758748284, number of calls of integrand is 200475
0 estimator trained
10 estimator trained
20 estimator trained
30 estimator trained
40 estimator trained
50 estimator trained
60 estimator trained
70 estimator trained
80 estimator trained
90 estimator trained
Current error is 4.095687504562164, number of calls of integrand is 198450
0 estimator trained
10 estimator trained
20 estimator trained
30 estimator trained
40 estimator trained
50 estimator trained
60 estimator trained
70 estimator trained
80 estimator trained
90 estimator trained
Current error is 3.7752352477250817, number of calls of integrand is 196425
0 estimator trained
10 estimator trained
20 estimator trained
30 estimator trained
40 estimator trained
50 estimator trained
60 estimator trained
7